# Pipeline to process medical reports

In this notebook, we will walkthrough on how to build a pipeline that will process medical reports in PDF format to extract relevant medical information by using the following AWS services:

- [Textract](https://aws.amazon.com/textract/): To extract text from the PDF medical report
- [Comprehend Medical](https://aws.amazon.com/comprehend/medical/): To extract relevant medical information from the output of textract

In [1]:
import boto3
import time
import sagemaker
import os 
from trp import Document
import pandas as pd

bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/medical_notes'

## Process PDF with Amazon Textract

In this section, we will be using Amazon Textract to extract text from a sample PDF medical record. If you're interested, you can preview the PDF report in the reports directory.

Amazon Textract can detect and analyze text in both single page and multi page documents. Single page document analysis can be performed using a Textract synchronous operation. However, multi page processing is an asynchronous operation. In this lab, we will be processing our medical report using the multi page processing method.

For more information on single page and multi page document processing, please refer to the following link:

- Single Page Processing: https://docs.aws.amazon.com/textract/latest/dg/sync.html
- Multi Page Processing: https://docs.aws.amazon.com/textract/latest/dg/async.html

As we are using the multi page textract operation, we will need to need upload our sample medical record to an S3 bucket. Run the next cell to upload our sample medical report.

In [2]:
fileName =  'sample_report_1.pdf'
fileUploadPath = os.path.join('./data', fileName)
textractObjectName = os.path.join(prefix, 'data', fileName)

# Upload medical report file
boto3.Session().resource('s3').Bucket(bucket).Object(textractObjectName).upload_file(fileUploadPath)

In the next step, we will start the asynchronous textract operation by calling the `start_document_analysis()` function. The function will kickoff an asynchronous job that will process our medical report file in the stipulated S3 bucket.

In [3]:
client = boto3.client('textract')
response = client.start_document_analysis(
    DocumentLocation={
        'S3Object': {
            'Bucket': bucket,
            'Name': textractObjectName
        }},
    FeatureTypes=[
        'TABLES',
    ]
    )

textractJobId = response["JobId"]

As the job is kicked off in the background, we can monitor the progress of the job by calling the `get_document_analysis()` function and passing the job id of the job that we created. 

Run the next cell and wait for the Textract Job status to return a SUCCEEDED status.

In [4]:
time.sleep(5)
response = client.get_document_analysis(JobId=textractJobId)
status = response["JobStatus"]
# print("Job status: {}".format(status))

while(status == "IN_PROGRESS"):
    time.sleep(5)
    response = client.get_document_analysis(JobId=textractJobId)
    status = response["JobStatus"]
    print("Textract Job status: {}".format(status))

Textract Job status: IN_PROGRESS
Textract Job status: SUCCEEDED


Now that we've successfully extracted the text from the medical report, let us extract the textract results and consolidate the text so that we can pass it to Comprehend Medical to start extract medical information from the report.

In [5]:
pages = []

time.sleep(5)

response = client.get_document_analysis(JobId=textractJobId)

pages.append(response)
print("Resultset page recieved: {}".format(len(pages)))
nextToken = None
if('NextToken' in response):
    nextToken = response['NextToken']

while(nextToken):
    time.sleep(5)

    response = client.get_document_analysis(JobId=textractJobId, NextToken=nextToken)

    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']

# Format the response from textract using the trp Document library
doc = Document(pages)

Resultset page recieved: 1


## Extract medical information with Amazon Comprehend Medical 

In the following section, we will be using Amazon Comprehend Medical to extract medical information from the text we got from the Textract operation on the PDF medical report.

With Amazon Comprehend Medical, you can perform the following on your documents:

- [Detect Entities (Version 2)](https://docs.aws.amazon.com/comprehend/latest/dg/extracted-med-info-V2.html) - Examine unstructured clinical text to detect textual references to medical information such as medical condition, treatment, tests and results, and medications. This version uses a new model and changes the way some entities are returned in the output. For more information, see [DetectEntitiesV2](https://docs.aws.amazon.com/comprehend/latest/dg/API_medical_DetectEntitiesV2.html).

- [Detect PHI](https://docs.aws.amazon.com/comprehend/latest/dg/how-medical-phi.html) —Examine unstructured clinical text to detect textual references to protected health information (PHI) such as names and addresses.


In this lab, we will be using the detect entities function to extract the medical condition. In the following cell, we will be processing the text on each page in batches of 20,000 UTF-8 characters. This is because Comprehend Medical has a maximum document size of 20,000 bytes (reference: https://docs.aws.amazon.com/comprehend/latest/dg/guidelines-and-limits-med.html). Once we've processed the text, we will then stich up the response into a into a single variable where we can either save to a csv or use for our analysis.

In [6]:
maxLength=20000

comprehendResponse = []

for page in doc.pages:
    pageText = page.text
    comprehend_medical_client = boto3.client(service_name='comprehendmedical', region_name='ap-southeast-2')
    for i in range(0, len(pageText), maxLength):
        response = comprehend_medical_client.detect_entities_v2(Text=pageText[0+i:maxLength+i])
        comprehendResponse.append(response)

In [7]:
def extract_medical_info(comprehendResponse):
    list_icd10=[]
    medical_conditions=[]
    scores=[]
    traits=[]
    for comprehendEntities in comprehendResponse:
        for row in comprehendEntities['Entities']:
            # 
            if row['Category'] == "MEDICAL_CONDITION":
                medical_conditions.append(row['Text'])#  += row['Text'] + ' '
                scores.append(row['Score'])
                trait='NaN'
                if row['Traits']:
                    #print(row['Traits'],row['Text'] )
                    trait=row['Traits'][0]['Name']

                traits.append(trait)
    
    df_mc = pd.DataFrame({'MEDICAL_CONDITION': pd.Series(medical_conditions), 'Score':pd.Series(scores),'Trait':pd.Series(traits)})
        
    return df_mc

In [8]:
# Format and extract medical information from comprehend medical response
medical_info = extract_medical_info(comprehendResponse)

medical_info

,MEDICAL_CONDITION,Score,Trait
0,jelly-like red-brown stool,0.370149,SYMPTOM
1,vomit,0.688994,NaN
2,hypertension,0.992857,DIAGNOSIS
3,B strep,0.455369,DIAGNOSIS
4,herpes,0.960876,DIAGNOSIS
...,...,...,...
62,vomiting,0.993813,SYMPTOM
63,burping,0.931319,SYMPTOM
64,gastroenteritis,0.991151,DIAGNOSIS
65,metabolic acidosis,0.933932,DIAGNOSIS


In [9]:
# Filter duplicates
medical_info_filtered = medical_info.sort_values(by=['MEDICAL_CONDITION']).drop_duplicates(['MEDICAL_CONDITION'],keep='last')

medical_info_filtered

,MEDICAL_CONDITION,Score,Trait
3,B strep,0.455369,DIAGNOSIS
28,CHEST motion,0.540621,SIGN
20,"Clear TMs, nose, and oropharynx",0.888015,SIGN
29,Clear to auscultation,0.985618,SIGN
40,GENITALIA Female genitalia is present on a visual,0.460017,SIGN
18,Grossly conjugate,0.688239,SIGN
45,Moves all extremities,0.723472,SIGN
48,NEUROLOGICALLY Eyes open,0.850665,SIGN
32,Regular,0.731455,SIGN
41,SK,0.399798,NaN
